In [ ]:
# Получение картинок шахматной доски для калибровки искажений
# https://habr.com/ru/articles/341160/
import numpy as np
import cv2

img_counter =2
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720) 

while(True): 
    ret, frame = cap.read()

    rows,cols,ch = frame.shape
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('s'):
        img_name = "chessboard_{}.png".format(img_counter)
        img_counter=img_counter+1
        cv2.imwrite(img_name, frame)
        print(img_counter)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
 

In [ ]:
# Получение коэфициентов трасформации картинок шахматной доски для калибровки искажений
# https://habr.com/ru/articles/341160/


from __future__ import print_function
import numpy as np
import cv2
from common import splitfn
import os

import sys
import getopt
from glob import glob

args, img_mask = getopt.getopt(sys.argv[1:], '', ['debug=', 'square_size='])
args = dict(args)
args.setdefault('--debug',  'C:\\Users\\HeroPC\\git\\Python\\WeedAI')
args.setdefault('--square_size', 1.0)
if not img_mask:
    img_mask = 'C:\\Users\\HeroPC\\git\\Python\\WeedAI\\*.png'
else:
    img_mask = img_mask[0]

img_names = glob(img_mask)
debug_dir = args.get('--debug')
if not os.path.isdir(debug_dir):
    os.mkdir(debug_dir)
square_size = float(args.get('--square_size'))

pattern_size = (9, 6)
pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
pattern_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size

obj_points = []
img_points = []
h, w = 0, 0
img_names_undistort = []
for fn in img_names:
    print('processing %s... ' % fn, end='')
    img = cv2.imread(fn, 0)
    if img is None:
        print("Failed to load", fn)
        continue

    h, w = img.shape[:2]
    found, corners = cv2.findChessboardCorners(img, pattern_size)
    if found:
        term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 0.1)
        cv2.cornerSubPix(img, corners, (5, 5), (-1, -1), term)

    if not found:
        print('chessboard not found')
        continue

    img_points.append(corners.reshape(-1, 2))
    obj_points.append(pattern_points)

    print('ok')

rms, camera_matrix, dist_coefs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, (w, h), None, None)

print("\nRMS:", rms)
print("camera matrix:\n", camera_matrix)
print("distortion coefficients: ", dist_coefs.ravel())

cv2.destroyAllWindows()

In [5]:
# https://docs.opencv.org/3.1.0/dc/dbb/tutorial_py_calibration.html

import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((9*6,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('C:\\Users\\HeroPC\\git\\Python\\WeedAI\\chessboard\\*.png')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
  
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [9]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

img = cv2.imread('C:\\Users\\HeroPC\\git\\Python\\WeedAI\\chessboard\\chessboard_1.png')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

cv2.imwrite('calibresult-undistort.png',dst)
 
# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult-undistort-crop.png',dst)


tot_error = 0
mean_error = 0
for i in  range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error
 
    print ("total error: ", mean_error/len(objpoints))

total error:  0.0
total error:  0.0
total error:  0.0
total error:  0.0
total error:  0.0
total error:  0.0
total error:  0.0
total error:  0.0
